In [78]:
import json
import numpy as np
from sklearn.model_selection import train_test_split
import random
import math
import tensorflow.keras as keras
import matplotlib.pyplot as plt
import tensorflow as tf

In [97]:
DATASET_PATH = "./client1.npy"

In [98]:
# def load_data(dataset_path):
#     with open(dataset_path,"r") as fp:
#         data = json.load(fp)
    
#     #convert lists into np arrays
    
#     X = np.array(data["mfcc"])
#     y = np.array(data["labels"])
    
#     return X,y

In [99]:
def load_data(dataset_path):
    with open(dataset_path,"rb") as f:
        X = np.load(f)
        y = np.load(f)
    return X,y

In [100]:
X,y = load_data(DATASET_PATH)
print(X.shape[0])

80


In [83]:
print(X[0])

[[-296.27041626  239.06048584 -117.58309937 ...   -1.98899078
    -6.46353006   -0.83947045]
 [-306.44400024  257.07836914 -116.79279327 ...    0.72331721
    -4.28372049   -5.11838961]
 [-305.78030396  263.25567627 -130.02761841 ...   -4.77825403
     2.17274904   -7.0064044 ]
 ...
 [-553.16247559  143.78601074  -33.30794907 ...   -4.72517395
    -6.33811283   -8.42058563]
 [-550.01843262  143.0435791   -42.74263763 ...   -3.2981925
    -9.43419075  -11.0783062 ]
 [-541.57073975  144.80627441  -53.12816238 ...    1.72554743
    -3.37020421   -6.60880756]]


In [84]:
print(y)

[1 0 1 0 0 0 1 1 1 1 1 0 0 0 0 1 1 1 1]


In [85]:
print(X[100:].shape)

(0, 40, 30)


In [86]:
def splitFunction(X,y,test_size,valid_size):
    test_samples = math.ceil(test_size*X.shape[0])
    valid_samples = math.ceil(valid_size*X.shape[0])
    
    X_test = X[0:test_samples]
    y_test = y[0:test_samples]
    X_valid = X[test_samples:valid_samples+test_samples]
    y_valid = y[test_samples:valid_samples+test_samples]
    X_train = X[valid_samples+test_samples:]
    y_train = y[valid_samples+test_samples:]
    
    return X_train, X_valid, X_test, y_train, y_valid, y_test
    

In [87]:
def prepare_datasets(test_size,valid_size):
    #load data
    X,y = load_data(DATASET_PATH)
    
    #create train test split
    X_new = []
    y_new = []
    
    j=0
    loop_limit = X.shape[0]
    while(j<=loop_limit-1):
        i = random.randint(0, X.shape[0]-1)
        X_new.append(X[i])
        y_new.append(y[i])
        
        X = np.delete(X, i, axis=0)
        y = np.delete(y, i, axis=0)

        j+=1
    
    X = np.array(X_new)
    y = np.array(y_new)
        
    #create train validation split
    
    X_train,X_valid,X_test,y_train,y_valid,y_test = splitFunction(X,y,test_size,valid_size)

    #make into 3D array
    
    X_train = X_train[...,np.newaxis]
    X_valid = X_valid[...,np.newaxis]
    X_test = X_test[...,np.newaxis]
    
    return X_train, X_valid, X_test, y_train, y_valid, y_test
    

In [88]:
X_train, X_valid, X_test, y_train, y_valid, y_test = prepare_datasets(0.2,0.25)

In [89]:
print(y.shape)

(19,)


In [90]:
print(y)

[1 0 1 0 0 0 1 1 1 1 1 0 0 0 0 1 1 1 1]


In [91]:
#Build model arch

In [92]:
def build_model(input_shape):
    model = keras.Sequential()

    # 1st conv layer
    model.add(keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(keras.layers.MaxPooling2D((3, 3), strides=(2, 2), padding='same'))
    model.add(keras.layers.BatchNormalization())

    # 2nd conv layer
    model.add(keras.layers.Conv2D(32, (3, 3), activation='relu'))
    model.add(keras.layers.MaxPooling2D((3, 3), strides=(2, 2), padding='same'))
    model.add(keras.layers.BatchNormalization())

    # 3rd conv layer
    model.add(keras.layers.Conv2D(32, (2, 2), activation='relu'))
    model.add(keras.layers.MaxPooling2D((2, 2), strides=(2, 2), padding='same'))
    model.add(keras.layers.BatchNormalization())

    # flatten output and feed it into dense layer
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(64, activation='relu'))
    model.add(keras.layers.Dropout(0.3))

    # output layer
    model.add(keras.layers.Dense(2, activation='softmax'))

    return model

In [93]:
input_shape = (X_train.shape[1],X_train.shape[2],X_train.shape[3])
model = build_model(input_shape)


In [94]:
optimiser = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimiser,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 38, 28, 32)        320       
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 19, 14, 32)        0         
_________________________________________________________________
batch_normalization_12 (Batc (None, 19, 14, 32)        128       
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 17, 12, 32)        9248      
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 9, 6, 32)          0         
_________________________________________________________________
batch_normalization_13 (Batc (None, 9, 6, 32)          128       
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 8, 5, 32)         

In [95]:
history = model.fit(X_train, y_train, validation_data=(X_valid, y_valid), batch_size=32, epochs=200)

Epoch 1/200
1/1 [==============================] - 0s 475ms/step - loss: 1.2751 - accuracy: 0.3000 - val_loss: 0.0140 - val_accuracy: 1.0000
Epoch 2/200
1/1 [==============================] - 0s 21ms/step - loss: 1.6240 - accuracy: 0.3000 - val_loss: 0.0359 - val_accuracy: 1.0000
Epoch 3/200
1/1 [==============================] - 0s 21ms/step - loss: 1.1324 - accuracy: 0.2000 - val_loss: 0.0584 - val_accuracy: 1.0000
Epoch 4/200
1/1 [==============================] - 0s 22ms/step - loss: 1.2046 - accuracy: 0.4000 - val_loss: 0.0779 - val_accuracy: 1.0000
Epoch 5/200
1/1 [==============================] - 0s 22ms/step - loss: 0.7499 - accuracy: 0.7000 - val_loss: 0.0982 - val_accuracy: 1.0000
Epoch 6/200
1/1 [==============================] - 0s 22ms/step - loss: 0.6391 - accuracy: 0.6000 - val_loss: 0.1186 - val_accuracy: 1.0000
Epoch 7/200
1/1 [==============================] - 0s 23ms/step - loss: 0.6313 - accuracy: 0.7000 - val_loss: 0.1377 - val_accuracy: 1.0000
Epoch 8/200
1/1 [==

1/1 [==============================] - 0s 21ms/step - loss: 0.0327 - accuracy: 1.0000 - val_loss: 0.4129 - val_accuracy: 1.0000
Epoch 59/200
1/1 [==============================] - 0s 21ms/step - loss: 0.0565 - accuracy: 1.0000 - val_loss: 0.4142 - val_accuracy: 1.0000
Epoch 60/200
1/1 [==============================] - ETA: 0s - loss: 0.0260 - accuracy: 1.00 - 0s 23ms/step - loss: 0.0260 - accuracy: 1.0000 - val_loss: 0.4151 - val_accuracy: 1.0000
Epoch 61/200
1/1 [==============================] - 0s 21ms/step - loss: 0.0508 - accuracy: 1.0000 - val_loss: 0.4160 - val_accuracy: 1.0000
Epoch 62/200
1/1 [==============================] - 0s 21ms/step - loss: 0.0270 - accuracy: 1.0000 - val_loss: 0.4165 - val_accuracy: 1.0000
Epoch 63/200
1/1 [==============================] - 0s 20ms/step - loss: 0.0418 - accuracy: 1.0000 - val_loss: 0.4168 - val_accuracy: 1.0000
Epoch 64/200
1/1 [==============================] - 0s 20ms/step - loss: 0.0293 - accuracy: 1.0000 - val_loss: 0.4168 - val_a

Epoch 116/200
1/1 [==============================] - 0s 20ms/step - loss: 0.0152 - accuracy: 1.0000 - val_loss: 0.4702 - val_accuracy: 0.6000
Epoch 117/200
1/1 [==============================] - 0s 21ms/step - loss: 0.0383 - accuracy: 1.0000 - val_loss: 0.4709 - val_accuracy: 0.6000
Epoch 118/200
1/1 [==============================] - 0s 22ms/step - loss: 0.0126 - accuracy: 1.0000 - val_loss: 0.4718 - val_accuracy: 0.6000
Epoch 119/200
1/1 [==============================] - 0s 19ms/step - loss: 0.0147 - accuracy: 1.0000 - val_loss: 0.4726 - val_accuracy: 0.6000
Epoch 120/200
1/1 [==============================] - 0s 23ms/step - loss: 0.0064 - accuracy: 1.0000 - val_loss: 0.4734 - val_accuracy: 0.6000
Epoch 121/200
1/1 [==============================] - 0s 22ms/step - loss: 0.0108 - accuracy: 1.0000 - val_loss: 0.4743 - val_accuracy: 0.6000
Epoch 122/200
1/1 [==============================] - 0s 22ms/step - loss: 0.0151 - accuracy: 1.0000 - val_loss: 0.4750 - val_accuracy: 0.6000
Epoch 

Epoch 174/200
1/1 [==============================] - 0s 23ms/step - loss: 0.0067 - accuracy: 1.0000 - val_loss: 0.4986 - val_accuracy: 0.6000
Epoch 175/200
1/1 [==============================] - 0s 23ms/step - loss: 0.0233 - accuracy: 1.0000 - val_loss: 0.4987 - val_accuracy: 0.6000
Epoch 176/200
1/1 [==============================] - 0s 20ms/step - loss: 0.0158 - accuracy: 1.0000 - val_loss: 0.4989 - val_accuracy: 0.6000
Epoch 177/200
1/1 [==============================] - 0s 19ms/step - loss: 0.0034 - accuracy: 1.0000 - val_loss: 0.4992 - val_accuracy: 0.6000
Epoch 178/200
1/1 [==============================] - 0s 22ms/step - loss: 0.0045 - accuracy: 1.0000 - val_loss: 0.4994 - val_accuracy: 0.6000
Epoch 179/200
1/1 [==============================] - 0s 20ms/step - loss: 0.0037 - accuracy: 1.0000 - val_loss: 0.4996 - val_accuracy: 0.6000
Epoch 180/200
1/1 [==============================] - 0s 21ms/step - loss: 0.0081 - accuracy: 1.0000 - val_loss: 0.4999 - val_accuracy: 0.6000
Epoch 

In [96]:
#load data
#split into train and test sets
#build network arch
#compile network
#train network